<a href="https://colab.research.google.com/github/arthur-siqueira/metodos-numericos/blob/main/eliminacao_gaussiana.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import scipy.linalg as sla

In [2]:
# Função de substituição retroativa

def substituicao_retroativa(A_escalonada, b_escalonado):
  """
  Resolve um sistema linear superior triangular.
  Recebe: Matriz A (triangular superior) e vetor b.
  Retorna: Vetor solução x.
  """

  n = len(b_escalonado)
  x = np.zeros(n)

  # Percorre as linhas de baixo para cima (n-1 até 0)
  for i in range(n - 1, -1, -1):
      # Calcula a soma dos termos já conhecidos (A[i, j] * x[j])
      soma = np.dot(A_escalonada[i, i+1:], x[i+1:])

      # Calcula o valor da variável x[i]
      x[i] = (b_escalonado[i] - soma) / A_escalonada[i, i]

  return x

In [3]:
# Função de eliminação gaussiana completa

def eliminacao_gaussiana_solver(A, b):
  # Cria a matriz estendida
    Ab = np.c_[A.astype(float), b.astype(float)] # Garante que os tipos são float para divisões
    n = len(A)

    # Processo de ESCALONAMENTO (Eliminação)
    for c in range(n - 1): # c é o índice da coluna pivô
        # Implementação de Pivotamento Parcial (Para maior robustez - Boa prática)
        # Encontra o índice da linha com o maior valor absoluto na coluna c, de c para baixo
        p = np.abs(Ab[c:, c]).argmax() + c

        # Se o pivô não for o elemento Ab[c,c], troca as linhas c e p
        if p != c:
            Ab[[c, p]] = Ab[[p, c]]
            # Verifica se o pivô é zero (o sistema pode não ter solução única)
        if Ab[c, c] == 0:
            raise ValueError(f"O sistema não pode ser resolvido por este método: O pivô na coluna {c} é zero após o pivotamento.")

        # Faz a eliminação para as linhas l abaixo da linha c
        for l in range(c + 1, n):
            # Fator multiplicador (m)
            m = Ab[l, c] / Ab[c, c]
            # Nova linha l = Linha l - m * Linha pivô (c)
            Ab[l] = Ab[l] - m * Ab[c]

    # Extrai a nova matriz triangular superior e o vetor b
    A_escalonada = Ab[:, :-1]
    b_escalonado = Ab[:, -1]
    # Processo de SUBSTITUIÇÃO RETROATIVA
    x = substituicao_retroativa(A_escalonada, b_escalonado)

    return x

In [5]:
# Definição dos sistemas de equações

sistemas = {
    "(a)": {
        "A": np.array([[1, 1, 1], [4, 4, 2], [2, 1, -1]]),
        "b": np.array([1, 2, 0])
    },
    "(b)": {
        "A": np.array([[7, -7, 2], [-3, 1, 7], [7, 7, -72]]),
        "b": np.array([1, 2, 7])
    },
    "(c)": {
        "A": np.array([[1, 2, 3, 4], [2, 1, 3, 4], [3, 1, 3, 4], [4, 1, 4, 3]]),
        "b": np.array([20, 22, 22, 24])
    }
}

print("--- Comparação dos Resultados (Arthur's Solver vs. Scipy Solve) ---\n")

for nome, sistema in sistemas.items():
    A = sistema["A"]
    b = sistema["b"]

    # Tenta resolver com a função implementada
    try:
        solucao_propria = eliminacao_gaussiana_solver(A, b)

        # Resolve com a função da biblioteca Scipy para comparação
        solucao_scipy = sla.solve(A, b)

        # Verifica a diferença entre as soluções
        erro = np.linalg.norm(solucao_propria - solucao_scipy)

        print(f"Sistema {nome}:")
        print(f"  Solução Própria (x): {solucao_propria}")
        print(f"  Solução Scipy (x):   {solucao_scipy}")
        print(f"  Erro Absoluto (Norma): {erro:.2e}\n")

    except ValueError as e:
        print(f"Sistema {nome}: FALHA na solução própria. {e}\n")
    except Exception as e:
        print(f"Sistema {nome}: FALHA na solução Scipy/Outro erro. Erro: {e}\n")

--- Comparação dos Resultados (Arthur's Solver vs. Scipy Solve) ---

Sistema (a):
  Solução Própria (x): [ 1. -1.  1.]
  Solução Scipy (x):   [ 1. -1.  1.]
  Erro Absoluto (Norma): 0.00e+00

Sistema (b):
  Solução Própria (x): [-5.48120301 -5.96992481 -1.21052632]
  Solução Scipy (x):   [-5.48120301 -5.96992481 -1.21052632]
  Erro Absoluto (Norma): 2.00e-15

Sistema (c):
  Solução Própria (x): [ 0.         -2.          4.57142857  2.57142857]
  Solução Scipy (x):   [ 2.27595720e-15 -2.00000000e+00  4.57142857e+00  2.57142857e+00]
  Erro Absoluto (Norma): 4.68e-15

